In [13]:
from utils import watershed_from_boundary_distance
import os 
import numpy as np 
import napari
import torch
from skimage.io import imread

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import numpy as np
assert torch.cuda.is_available()
from torchmetrics.classification import Dice, MulticlassAccuracy
from model3d import Unet3D

from dataset_dst import (
    BlastoDataset
)

from model3d import Unet3D

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

if device is None:
    # You can pass in a device or we will default to using
    # the gpu. Feel free to try training on the cpu to see
    # what sort of performance difference there is
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")


In [32]:


model_path = '/localscratch/DL4MIA_2024/BlastoSeg/saved_models/unet3d_model_stepsize2_best.pth'
checkpoint = torch.load(model_path)
model = Unet3D(n_classes=2)
model.load_state_dict(checkpoint)
model.eval()
# model.to(device)



Unet3D(
  (dconv_down1): Sequential(
    (0): Conv3d(1, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
    (1): ReLU(inplace=True)
    (2): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
    (3): ReLU(inplace=True)
  )
  (dconv_down2): Sequential(
    (0): Conv3d(64, 128, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
    (1): ReLU(inplace=True)
    (2): Conv3d(128, 128, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
    (3): ReLU(inplace=True)
  )
  (dconv_down3): Sequential(
    (0): Conv3d(128, 256, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
    (1): ReLU(inplace=True)
    (2): Conv3d(256, 256, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
    (3): ReLU(inplace=True)
  )
  (dconv_down4): Sequential(
    (0): Conv3d(256, 512, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
    (1): ReLU(inplace=True)
    (2): Conv3d(512, 512, kernel_size=(1, 3, 3), stride=(1, 1, 1), paddi

In [28]:
viewer = napari.Viewer()

In [33]:
val_data = BlastoDataset("/group/dl4miacourse/projects/BlastoSeg/validation")
val_loader = DataLoader(val_data, batch_size = 1, shuffle=False, num_workers=8)

step_size = 118
with torch.no_grad():
    for x_batch, y_batch, _ in val_loader: 
        print(x_batch.shape, y_batch.shape)
        z_slices = x_batch.shape[1]
        num_iterations = int(z_slices / step_size)
        loss = 0
        for i in range(0,num_iterations,step_size): 

            start_index = i
            end_index = min(start_index + step_size, x_batch.size(2))

            x = x_batch[:, start_index:end_index, :, :] # [1,2,256,256] BDHW
            x = torch.unsqueeze(x, 0) # [1,1,2,256,256] BCDHW

            y = y_batch[:, start_index:end_index, :, :]
            y = torch.unsqueeze(y, 0)

            prediction = model(x)  # Assuming model expects a batch dimension
    
            if y.dtype != prediction.dtype:
                y = y.type(prediction.dtype)
            
            viewer.add_image(prediction.numpy())




torch.Size([1, 118, 256, 256]) torch.Size([1, 118, 256, 256])
torch.Size([1, 118, 256, 256]) torch.Size([1, 118, 256, 256])


In [9]:
viewer = napari.Viewer()

In [11]:

pred = model(img)

viewer.add_image(img)
viewer.add_image(pred)

# perform watershed to calculate instances from distance map predictions
seeds, seg = watershed_from_boundary_distance(pred, pred>0)
viewer.add_image(seeds)
viewer.add_labels(seg)

TypeError: conv3d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int, int)!, !tuple of (int, int, int)!, !tuple of (int, int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int, int)!, !tuple of (int, int, int)!, !tuple of (int, int, int)!, int)
